In [1]:
import numpy as np
import torch
from torch import nn,optim
from torch.autograd import Variable
from torchvision import datasets,transforms
from torch.utils.data import DataLoader

In [2]:
#训练集
train_dataset=datasets.MNIST(root='./',#等号右侧为数据集的路径,./表示当前路径
                             train=True,#
                             transform=transforms.ToTensor(),#将MNIST数据集中数据转化为tensor数据
                             download=True#若路径不存在数据集,则下载数据
)
#测试集
test_dataset=datasets.MNIST(root='./',
                             train=False,
                             transform=transforms.ToTensor(),
                             download=True
)

In [22]:
#批次大小
batch_size=64
#装载数据集
train_loader=DataLoader(dataset=train_dataset,
                        batch_size=batch_size,
                        shuffle=True#打乱顺序
)
test_loader=DataLoader(dataset=test_dataset,
                        batch_size=batch_size,
                        shuffle=True#打乱顺序
)

In [24]:
#定义网络结构
class MnistNet(nn.Module):
    def __init__(self):
        super(MnistNet,self).__init__()
        #全连接层1
        #对于 MNIST 数据集，图像尺寸是 28x28，因此输入特征数是 784（即 28*28）
        self.fc1 = nn.Linear(28*28,28)#此处第二个28为输出特征数,也可以为其他数字,但是要确保与下一层输入特征数相同
        #全连接层2
        self.fc2 = nn.Linear(28,10)#输出特征数为10,因为结果有0`9十个可能

    def forward(self,x):
        # 输入层,全连接层的输入需要一维向量,因此将输入展开为以为
        x = x.view(-1,28*28*1)
        # 隐藏层 
        x = self.fc1(x)
        # 隐藏层使用relu作为激活函数
        x = nn.functional.relu(x)
        # 输出层
        x = self.fc2(x) 
        # 输出层使用softmax激活函数
        return nn.functional.log_softmax(x,dim=-1)
        

In [25]:
#定义学习率
LR=0.5
#实例化模型
model=MnistNet()
#定义损失函数
mse_loss=nn.MSELoss()
#定义优化器
optimizer=optim.SGD(model.parameters(),LR)

In [26]:
#定义训练函数
def train():
    for i,data in enumerate(train_loader):
        #循环获得一个批次的数据和标签
        inputs,labels=data
        #获得模型预测结果(64,10)
        outputs=model(inputs)
        #将数据标签转化为独热编码 
        #(64)->(64,1)
        labels=labels.reshape(-1,1)#同样,此处的-1表示自动匹配,在此应为64
        #tensor.scatter(dim,index,src)
        #(64,1)->(64,10)
        one_hot=torch.zeros(inputs.shape[0],10).scatter(1,labels,1)
        #计算损失函数
        loss=mse_loss(outputs,one_hot)
        #梯度清零
        optimizer.zero_grad()
        #计算梯度
        loss.backward()
        #修改权值
        optimizer.step()
def test():
    correct=0
    for i,data in enumerate(test_loader):
        #获得一个批次的数据和标签
        inputs,labels=data
        #获得模型预测结果 (64,10)
        outputs=model(inputs)
        #获得最大值,以及最大值所在位置
        _,label_hat=torch.max(outputs,1)
        #预测正确率
        #if((label_hat==label):
        #correct+=1
        correct += (label_hat == labels).sum().item()  # 修正逻辑错误：直接比较并求和
    print("Test acc: {0}".format(correct / len(test_dataset)))

In [27]:
#设置训练轮次为10
epoch=10
print("开始训练")
for i in range(epoch):
    print('epoch:',i)
    train()
    test()
print("结束训练")

开始训练
epoch: 0
Test acc: 0.9177
epoch: 1
Test acc: 0.9234
epoch: 2
Test acc: 0.9278
epoch: 3
Test acc: 0.9264
epoch: 4
Test acc: 0.9317
epoch: 5
Test acc: 0.9325
epoch: 6
Test acc: 0.9307
epoch: 7
Test acc: 0.9363
epoch: 8
Test acc: 0.9371
epoch: 9
Test acc: 0.9376
